# Monitoring Tasks

In [1]:
# Following code is needed to preconfigure this notebook
import datetime
import sys
import os
sys.path.insert(0, os.path.abspath('../../..'))

import pyflow as pf

## Labels

Sometime it is very useful to see task-specific information in GUI. For this, we can define a label. A label is a string that is attached to a task and that can be updated using the child command `ecflow_client --label`.

In [2]:
with pf.Suite('test', host=pf.LocalHost(), files='/test') as s:
    with pf.Family('label'):
        with pf.Task('t1', script=[
            'n=1',
            'while [[ $n -le 5 ]]                   # Loop 5 times',
            'do',
            '    msg=\"The date is now $(date)\"',
            '    ecflow_client --label=info \"$msg\"  # Set the label',
            '    sleep 60                           # Wait a one minute',
            '    (( n = $n + 1 ))',
            'done',
            '',
            'ecflow_client --label=info \"I have now finished my work.\"',
        ]) as t1:
            pf.Label('info', '')

s

Suite(/test)

In [3]:
s.deploy_suite(pf.Notebook)

## Late

In [4]:
with pf.Suite('test') as s:
    with pf.Family('f1'):
        with pf.Task('t1'):
            pf.Late('-s +00:15 -a 20:00 -c +02:00')

s

Suite(/test)

This is interpreted as: the node can stay submitted for a maximum of 15 minutes, and it must become active by 20:00 and the runtime must not exceed 2 hours.

In [5]:
with pf.Suite('test') as s:
    with pf.Family('f6'):
        pf.Variable('SLEEP', 120)
        with pf.Task('t1'):
            pf.Late('-c +00:01') # set late flag if task take longer than a minute

s

Suite(/test)

In [6]:
with pf.Suite('test', host=pf.LocalHost(), files='/test') as s:
    with pf.Family('label'):
        with pf.Task('t1', script=[
            'n=1',
            'while [[ $n -le 5 ]]                   # Loop 5 times',
            'do',
            '    msg="The date is now $(date)"',
            '    ecflow_client --label=info "$msg"  # Set the label',
            '    sleep 60                           # Wait a one minute',
            '    (( n = $n + 1 ))',
            'done',
            '',
            'ecflow_client --label=info "I have now finished my work."',
        ]) as t1:
            pf.Label('info', '')

s

Suite(/test)

## Meters

A meter is very similar to an event. Instead of being a boolean value (on/off), it can take a range of integer values. Other tasks are then triggered when the meter reaches a certain value.

Like events, meters have names and a task can have several of them.

In [7]:
with pf.Suite('test', host=pf.LocalHost(), files='/test') as s:
    with pf.Family('f1'):
        pf.Variable('SLEEP', 20)
        with pf.Task('t1') as t1:
            pf.Meter('progress', 1, 100, 90)
        with pf.Task('t2', script=[
            'echo "I will now sleep for %SLEEP% seconds"',
            'sleep %SLEEP%',
            'n=1',
            'while [[ $n -le 100 ]]                   # Loop 100 times',
            'do',
            '    sleep 1                              # Wait a short time',
            '    ecflow_client --meter=progress $n    # Notify ecFlow',
            '    (( n = $n + 1 ))',
            'done',
        ]) as t2:
            pf.Event('a')
            pf.Event('b')
        t3 = pf.Task('t3')
        t4 = pf.Task('t4')
        t5 = pf.Task('t5')
        t6 = pf.Task('t6')
        t7 = pf.Task('t7')
        
    t2.triggers = t1
    t3.triggers = t2.a
    t4.completes = t2.b
    t4.triggers = t2
    t5.triggers = t1.progress >= 30
    t6.triggers = t1.progress >= 60
    t7.triggers = t1.progress >= 90
    
s

Suite(/test)